# Predict Dataset (AFF and NFF)

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Define class labels
class_labels = ['AFF', 'NFF']

# Load the saved model
model = models.resnet18(weights=None)  # Instantiate the ResNet18 model
model.fc = nn.Linear(512, 2)  # Modify the output layer
model.load_state_dict(torch.load('resnet18_xray_classifier.pth'))  # Load the trained weights
model.eval()  # Set the model to evaluation mode

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 pixels
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

# Define the path to the directory containing your dataset
dataset_path = '/local/data1/honzh073/model/data/8bit_dataset'  # Replace with the actual path

# Create an ImageFolder dataset for the dataset
dataset = ImageFolder(root=dataset_path, transform=transform)

# Create a DataLoader for the dataset
batch_size = 64
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Initialize variables for accuracy calculation
correct_aff = 0
correct_nff = 0
total_aff = 0
total_nff = 0

# Make predictions and calculate accuracy
with torch.no_grad():
    for images, labels in data_loader:
        outputs = model(images)
        _, predicted_class = outputs.max(1)
        
        # Calculate accuracy for AFF (class 0)
        aff_indices = (predicted_class == 0).nonzero()
        correct_aff += (predicted_class[aff_indices] == labels[aff_indices]).sum().item()
        total_aff += aff_indices.size(0)
        
        # Calculate accuracy for NFF (class 1)
        nff_indices = (predicted_class == 1).nonzero()
        correct_nff += (predicted_class[nff_indices] == labels[nff_indices]).sum().item()
        total_nff += nff_indices.size(0)

# Calculate accuracy for AFF and NFF
accuracy_aff = 100 * correct_aff / total_aff
accuracy_nff = 100 * correct_nff / total_nff

# Print the results
print(f"Accuracy on AFF images: {accuracy_aff:.2f}%")
print(f"Accuracy on NFF images: {accuracy_nff:.2f}%")


Accuracy on AFF images: 95.68%
Accuracy on NFF images: 98.59%


# Single Image Classify

In [15]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

# Define class labels
class_labels = ['AFF', 'NFF']

# Load the saved model
model = models.resnet18(pretrained=False)  # Instantiate the ResNet18 model
model.fc = nn.Linear(512, 2)  # Modify the output layer
model.load_state_dict(torch.load('resnet18_xray_classifier.pth'))  # Load the trained weights
model.eval()  # Set the model to evaluation mode

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 pixels
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

# Load and preprocess an input image
# input_image_path = '/local/data1/honzh073/model/data/8bit_images/patient_BINZEEEHSR_hospital_55_AFF_image_4.png'
input_image_path = '/local/data1/honzh073/model/data/8bit_images/patient_ZHAIJPEBDR_hospital_21_AFF_image_4.png'

input_image = Image.open(input_image_path).convert('RGB')
input_tensor = transform(input_image).unsqueeze(0)  # Add a batch dimension

# Make a prediction
with torch.no_grad():
    output = model(input_tensor)
    _, predicted_class = output.max(1)

# Get the predicted class label
predicted_label = class_labels[predicted_class]

# Print the result
print(f"Predicted Class: {predicted_label}")

# check this image must
# /local/data1/honzh073/model/data/8bit_images/patient_ZHAIJPEBDR_hospital_21_AFF_image_4.png


Predicted Class: AFF
